In [2]:
# Dependencies
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import io
import time
from datetime import datetime, timedelta
import json
import pprint
pp = pprint.PrettyPrinter(indent=4)
pd.set_option('display.max_columns', None)

rFolder = "C:/Users/janin/Downloads/"
dataFolder = "D:/Repositories/Global-COVID-Surveillance/data/"
cleanedFolder = dataFolder + "cleaned/"

def print_column_unique(column):
    print("Column Values:")
    values = list(set(sorted(column,key=lambda v: (isinstance(v, str), v))))
    print(values)
    return values

def print_column_missing(column, comparison):
    values = print_column_unique(column)
    print("Comparison:")
    print(comparison)
    missing_values = []
    for value in values:
        inComparison = False
        for c in comparison:
            inComparison = (str(value) == str(c))
            if inComparison:
                break
        if not inComparison:
            missing_values.append(value)
    if len(missing_values) > 0:
        print("Column values not in comparison:")
        print(missing_values)
    else:
        print("No missing values")
    missing_values = []
    for value in comparison:
        if not value in values:
            missing_values.append(value)
    if len(missing_values) > 0:
        print("Comparison values not in column:")
        print(missing_values)
    else:
        print("No missing values")
    return values

now = datetime.now()
current_date = now.strftime('%Y-%m-%d')
#file_end = " -Results- " + current_date +" .csv"
file_end = " -Results- " + "2020-12-28" +" .csv"
print(file_end)

 -Results- 2020-12-28 .csv


In [3]:
populations = pd.read_excel(cleanedFolder + "all_populations.xlsx")
population_groups = ["Level","Region","Census Region","Country","State/Province"]
for group in population_groups:
    populations[group] = populations[group].astype(str)
    populations[group] = populations[group].apply(lambda x: "" if x=="nan" else x)
populations.head()

,Level,Region,Country,Census Region,State/Province,Population,Population 100K,Country Population,Country Population 100K,Region Population,Region Population 100K,World Population,World Population 100K,Country Share,Region Share,World Share,World Share (%),Urban Population (%),Annual Change (%),Net Change,Migrants (net),Density (P/Km²),Land Area (Km²),Fertility Rate,Median Age,< 1,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85+,1-4,5-14,15-24,25-34,35-44,45-54,55-64,65-74,75-84,Pct < 1,Pct 1-4,Pct 5-14,Pct 15-24,Pct 25-34,Pct 35-44,Pct 45-54,Pct 55-64,Pct 65-74,Pct 75-84,Pct 85+
0,Country,Central Asia,Armenia,,,2963243,29.63243,NaN,NaN,326887719,3268.87719,7796609105,77966.09105,NaN,0.009065,0.000380,0.04,63.0,0.19,5512.0,-4998.0,104.0,28470.0,1.8,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Country,Central Asia,Azerbaijan,,,10139177,101.39177,NaN,NaN,326887719,3268.87719,7796609105,77966.09105,NaN,0.031017,0.001300,0.13,56.0,0.91,91459.0,1200.0,123.0,82658.0,2.1,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Country,Central Asia,Cyprus,,,1207359,12.07359,NaN,NaN,326887719,3268.87719,7796609105,77966.09105,NaN,0.003693,0.000155,0.02,67.0,0.73,8784.0,5000.0,131.0,9240.0,1.3,37.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Country,Central Asia,Faeroe Islands,,,48863,0.48863,NaN,NaN,326887719,3268.87719,7796609105,77966.09105,NaN,0.000149,0.000006,0.00,43.0,0.38,185.0,NaN,35.0,1396.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Country,Central Asia,Georgia,,,3989167,39.89167,NaN,NaN,326887719,3268.87719,7796609105,77966.09105,NaN,0.012203,0.000512,0.05,58.0,-0.19,-7598.0,-10000.0,57.0,69490.0,2.1,38.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Read starter

pgmm = pd.read_excel(dataFolder + "Starter.xlsx")
print(pgmm.columns)
pgmm.head()

# Read R results

pgmm_files = [
    'Central Asia',
    'East Asia and Pacific',
    'Europe',
    'Latin America',
    'Middle East and North Africa',
    'United States',
    'Canada',
    'South Asia',
    'Sub-Saharan Africa'
]

for r in pgmm_files:
    pgmm_read_file = cleanedFolder + r + file_end
    df = pd.read_csv(pgmm_read_file)
    df = df.drop(["Unnamed: 0"], axis=1)
    df = df[df["V1"] != 0]
    if ((r == "Canada") or (r == "United States")):
        df["Region"] = "North America"
        df.rename(columns = {"V2": "State/Province"}, inplace=True)
        df["Country"] = r
        df["Level"] = df["State/Province"].apply(lambda x: "Country" if (x == "Region") else "State/Province")
        df["State/Province"] = df["State/Province"].apply(lambda x: "" if (x == "Region") else x)
    else:
        df["Region"] = r
        df.rename(columns = {"V2": "Country"}, inplace=True)
        df["Level"] = df["Country"].apply(lambda x: "Region" if (x == "Region") else "Country")
        df["Country"] = df["Country"].apply(lambda x: "" if (x == "Region") else x)
        df["State/Province"] = ""
    df.rename(columns = {
        "V1": 'Last Day of Week Excel Date',
        "V3": 'Cases Daily Last Day of Week',
        "V4": 'Total Cases Last Day of Week',
        "V5": 'Cases 7D Moving Average',
        "V6": 'Cases Last Day of Week Rate 100K',
        "V7": 'Deaths Daily Last Day of Week',
        "V8": 'Total Deaths Last Day of Week',
        "V9": 'Deaths 7D Moving Average',
        "V10": 'Deaths Last Day of Week Rate 100K',
        "V11": 'Speed', #Cases Last Day of Week Rate 100K 7D Moving Average
        "V12": 'Acceleration', #Average Daily Change in Speed
        "V13": 'Jerk', #Average Daily Change in Acceleration
        "V14": '1-Day Persistence', 
        "V15": '7-Day Persistence'
#        "Level": "Statistics Level",
#        "Region": "Statistics Region",
#        "Country": "Statistics Country",
#        "State/Province": "Statistics State/Province"
    }, inplace=True)
    pgmm = pd.concat([pgmm, df], ignore_index=True, sort=False)

# Create Time Variables
pgmm["Last Day of Week Excel Date"] = pgmm["Last Day of Week Excel Date"].astype(int)
pgmm["Time"] = pgmm["Last Day of Week Excel Date"].apply(lambda x: datetime.fromordinal(datetime(1900, 1, 1).toordinal() + x - 2))
pgmm["Date"] = pgmm["Time"].apply(lambda x: x.strftime('%m/%d/%Y'))
pgmm["Week"] = pgmm["Time"].apply(lambda x: x.strftime('%Y W%V'))
pgmm["Output"] = True
pgmm_order = [
    'Region', 'Country', 'State/Province', 
    'Last Day of Week Excel Date', 'Date', 'Week',
    'Speed', 'Acceleration', 'Jerk', '7-Day Persistence', '1-Day Persistence',
    'Cases Daily Last Day of Week', 'Total Cases Last Day of Week',
    'Cases 7D Moving Average', 'Cases Last Day of Week Rate 100K',
    'Deaths Daily Last Day of Week', 'Total Deaths Last Day of Week',
    'Deaths 7D Moving Average', 'Deaths Last Day of Week Rate 100K', 'Output'
]
pgmm = pgmm[pgmm_order]
pgmm.head()

Index(['Level', 'Region', 'Country', 'State/Province', 'Excel Date'], dtype='object')


,Region,Country,State/Province,Last Day of Week Excel Date,Date,Week,Speed,Acceleration,Jerk,7-Day Persistence,1-Day Persistence,Cases Daily Last Day of Week,Total Cases Last Day of Week,Cases 7D Moving Average,Cases Last Day of Week Rate 100K,Deaths Daily Last Day of Week,Total Deaths Last Day of Week,Deaths 7D Moving Average,Deaths Last Day of Week Rate 100K,Output
0,Central Asia,Armenia,,44149,11/14/2020,2020 W46,55.952212,-4.840257,-2.396024,10.539494,10.131422,1472.0,115855.0,1658.000000,49.675305,41.0,1738.0,29.285714,1.383619,True
1,Central Asia,Armenia,,44156,11/21/2020,2020 W47,43.311621,-1.345051,-1.171496,8.473356,7.442296,1193.0,124839.0,1283.428571,40.259945,31.0,1931.0,27.571429,1.046151,True
2,Central Asia,Armenia,,44163,11/28/2020,2020 W48,42.207618,0.265154,1.185959,6.559076,6.989958,1248.0,133594.0,1250.714286,42.116020,31.0,2121.0,27.142857,1.046151,True
3,Central Asia,Armenia,,44170,12/05/2020,2020 W49,35.506466,0.091598,1.499323,6.391886,5.902095,1267.0,140959.0,1052.142857,42.757209,32.0,2309.0,26.857143,1.079898,True
4,Central Asia,Armenia,,44177,12/12/2020,2020 W50,30.627641,-1.311305,-0.795461,6.323139,7.416970,995.0,147312.0,907.571429,33.578076,17.0,2462.0,21.857143,0.573696,True


In [5]:
us_check = pgmm.loc[(pgmm["Country"]=="United States") & (pgmm["State/Province"]=="")]
us_check.head()

,Region,Country,State/Province,Last Day of Week Excel Date,Date,Week,Speed,Acceleration,Jerk,7-Day Persistence,1-Day Persistence,Cases Daily Last Day of Week,Total Cases Last Day of Week,Cases 7D Moving Average,Cases Last Day of Week Rate 100K,Deaths Daily Last Day of Week,Total Deaths Last Day of Week,Deaths 7D Moving Average,Deaths Last Day of Week Rate 100K,Output
1127,North America,United States,,44150,11/15/2020,2020 W46,45.235231,1.500501,-0.042326,23.328169,4.843987,145344.0,10952685.0,131348.428571,44.440558,693.0,237073.0,1131.857143,0.211893,True
1128,North America,United States,,44157,11/22/2020,2020 W47,51.882654,0.302092,-0.388317,31.692237,5.712979,152260.0,12140470.0,148568.142857,46.555202,910.0,247417.0,1477.714286,0.278243,True
1129,North America,United States,,44164,11/29/2020,2020 W48,48.663076,-0.706570,0.666166,36.349485,5.468102,136084.0,13254547.0,137478.571429,41.609209,812.0,257431.0,1430.571429,0.248278,True
1130,North America,United States,,44171,12/06/2020,2020 W49,59.554815,1.919569,-0.939517,34.093818,6.383586,180030.0,14617976.0,173790.857143,55.046191,1138.0,272700.0,2181.285714,0.347956,True
1131,North America,United States,,44178,12/13/2020,2020 W50,64.133458,0.220454,-0.128158,40.116975,5.808863,185077.0,16086227.0,184138.000000,56.589368,1487.0,289631.0,2418.714286,0.454667,True


In [6]:
input_df = pd.read_excel(cleanedFolder + "all_raw_input.xlsx")
input_df.drop(columns=["MM-DD-YYYY","DD-MM-YYYY"],inplace=True)
input_df["State/Province"] = input_df["State/Province"].astype(str)
input_df["State/Province"] = input_df["State/Province"].apply(lambda x: "" if x == "nan" else x)
input_df["Census Region"] = input_df["Census Region"].astype(str)
input_df["Census Region"] = input_df["Census Region"].apply(lambda x: "" if x == "nan" else x)
input_df["Status"] = input_df["Status"].astype(str)
input_df["Status"] = input_df["Status"].apply(lambda x: "" if x == "nan" else x)
input_df["Data Quality"] = input_df["Data Quality"].astype(str)
input_df["Data Quality"] = input_df["Data Quality"].apply(lambda x: "" if x == "nan" else x)
input_df.head()

,Level,Region,Country,Census Region,State/Province,Abbreviation,FIPS,Time,Date,Week,Status,Data Quality,Cases Daily,Cases Daily 7D Rolling,Total Cases,Cases Daily Rate,Cases Daily Rate 7D Rolling,Total Cases Rate,Tests Daily,Tests Daily 7D Rolling,Total Tests,Tests Daily Rate,Tests Daily Rate 7D Rolling,Total Tests Rate,Deaths Daily,Deaths Daily 7D Rolling,Total Deaths,Deaths Daily Rate,Deaths Daily Rate 7D Rolling,Total Deaths Rate,Positivity 7D Rolling,Speed Daily,Acceleration Daily,Jerk Daily,Negative Daily,Total Negative,Hospitalized Daily,Total Hospitalized,Currently Hospitalized,ICU Daily,Total ICU,Currently In ICU,Total In ICU,Currently On Ventilator,Total On Ventilator,Recovered Daily,Total Recovered,Active Daily,Total Active,Population,Population 100K,Country Population,Country Population 100K,Country Share,Region Population,Region Population 100K,Region Share,World Population,World Population 100K,World Share,World Share (%),Urban Population (%),Annual Change (%),Net Change,Migrants (net),Density (P/Km²),Land Area (Km²),Fertility Rate,Median Age,< 1,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85+,1-4,5-14,15-24,25-34,35-44,45-54,55-64,65-74,75-84,Pct < 1,Pct 1-4,Pct 5-14,Pct 15-24,Pct 25-34,Pct 35-44,Pct 45-54,Pct 55-64,Pct 65-74,Pct 75-84,Pct 85+,Accessed
0,Country,Central Asia,Armenia,,,AM,NaN,2020-03-01,03/01/2020,2020 W09,,,1,NaN,1.0,0.033747,NaN,0.033747,0,NaN,0.0,0.0,NaN,0.0,0,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2963243.0,29.63243,NaN,NaN,NaN,326887719.0,3268.87719,0.009065,7.796609e+09,77966.09105,0.00038,0.04,63.0,0.19,5512.0,-4998.0,104.0,28470.0,1.8,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12/29/2020
1,Country,Central Asia,Armenia,,,AM,NaN,2020-03-02,03/02/2020,2020 W10,,,0,NaN,1.0,0.000000,NaN,0.033747,0,NaN,0.0,0.0,NaN,0.0,0,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2963243.0,29.63243,NaN,NaN,NaN,326887719.0,3268.87719,0.009065,7.796609e+09,77966.09105,0.00038,0.04,63.0,0.19,5512.0,-4998.0,104.0,28470.0,1.8,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12/29/2020
2,Country,Central Asia,Armenia,,,AM,NaN,2020-03-03,03/03/2020,2020 W10,,,0,NaN,1.0,0.000000,NaN,0.033747,0,NaN,0.0,0.0,NaN,0.0,0,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2963243.0,29.63243,NaN,NaN,NaN,326887719.0,3268.87719,0.009065,7.796609e+09,77966.09105,0.00038,0.04,63.0,0.19,5512.0,-4998.0,104.0,28470.0,1.8,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12/29/2020
3,Country,Central Asia,Armenia,,,AM,NaN,2020-03-04,03/04/2020,2020 W10,,,0,NaN,1.0,0.00

In [7]:
us_check = input_df.loc[(input_df["Country"]=="United States") & (input_df["State/Province"]=="")]
us_check.head()

,Level,Region,Country,Census Region,State/Province,Abbreviation,FIPS,Time,Date,Week,Status,Data Quality,Cases Daily,Cases Daily 7D Rolling,Total Cases,Cases Daily Rate,Cases Daily Rate 7D Rolling,Total Cases Rate,Tests Daily,Tests Daily 7D Rolling,Total Tests,Tests Daily Rate,Tests Daily Rate 7D Rolling,Total Tests Rate,Deaths Daily,Deaths Daily 7D Rolling,Total Deaths,Deaths Daily Rate,Deaths Daily Rate 7D Rolling,Total Deaths Rate,Positivity 7D Rolling,Speed Daily,Acceleration Daily,Jerk Daily,Negative Daily,Total Negative,Hospitalized Daily,Total Hospitalized,Currently Hospitalized,ICU Daily,Total ICU,Currently In ICU,Total In ICU,Currently On Ventilator,Total On Ventilator,Recovered Daily,Total Recovered,Active Daily,Total Active,Population,Population 100K,Country Population,Country Population 100K,Country Share,Region Population,Region Population 100K,Region Share,World Population,World Population 100K,World Share,World Share (%),Urban Population (%),Annual Change (%),Net Change,Migrants (net),Density (P/Km²),Land Area (Km²),Fertility Rate,Median Age,< 1,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85+,1-4,5-14,15-24,25-34,35-44,45-54,55-64,65-74,75-84,Pct < 1,Pct 1-4,Pct 5-14,Pct 15-24,Pct 25-34,Pct 35-44,Pct 45-54,Pct 55-64,Pct 65-74,Pct 75-84,Pct 85+,Accessed
43432,Country,North America,United States,,,US,NaN,2020-01-22,01/22/2020,2020 W04,,,0,0.285714,15.0,0.000000,0.000087,0.004586,10,74.142857,2593.0,0.003058,0.022670,0.792839,0,0.0,0.0,0.0,0.0,0.0,0.003854,1228.000000,-730.714286,-1023.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,327052602.0,3270.52602,NaN,NaN,NaN,365057840.0,3650.5784,0.895893,7.796609e+09,77966.09105,0.041948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3783052.0,3829599.0,3922044.0,3998665.0,4043323.0,4028281.0,4017227.0,4022319.0,4066194.0,4061874.0,4060940.0,4189261.0,4208387.0,4175221.0,4164459.0,4175459.0,4150420.0,4141202.0,4222496.0,4272909.0,4203852.0,4207331.0,4225571.0,4271168.0,4379797.0,4478029.0,4555029.0,4680384.0,4770188.0,4761228.0,4572098.0,4461418.0,4382364.0,4396137.0,4425400.0,4282329.0,4340779.0,4330751.0,4278041.0,4358571.0,4084430.0,4002096.0,3958280.0,3840006.0,3954899.0,3827009.0,3879028.0,4049147.0,4275506.0,4324119.0,4091962.0,4000602.0,3998899.0,4066500.0,4303847.0,4373250.0,4360049.0,4341663.0,4385025.0,4413491.0,4252458.0,4215037.0,4156618.0,3996074.0,3950547.0,3774597.0,3618069.0,3464437.0,3345475.0,3252423.0,3136704.0,3083083.0,3191048.0,2334433.0,2283164.0,2198286.0,2222392.0,1911261.0,1720817.0,1599909.0,1475278.0,1381641.0,1241341.0,1151190.0,1067757.0,6604958.0,15793631.0,40994163.0,42250205.0,45482275.0,41430182.0,40816619.0,42444212.0,31483433.0,15969872.0,0.011567,0.048291,0.125344,0.129185,0.139067,0.126677,0.124801,0.129778,0.096264,0.04883,0.020195,12/29/2020
43433,Country,North America,United States,,,US,NaN,2020-01-23,01/23/2020,2020 W04,,,0,0.285714,15.0,0.000000,0.000087,0.004586,36,103.142857,2850.0,0.011007,0.031537,0.871419,0,0.0,0.0,0.0,0.0,0.0,0.002770,1394.857143,236.142857,391.571429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,327052602.0,3270.52602,NaN,NaN,NaN,365057840.0,3650.5784,0.895893,7.796609e+09,77966.09105,0.041948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3783052.0,3829599.0,3922044.0,3998665.0,4043323.0,4028281.0,4017227.0,4022319.0,4066194.0,4061874.0,4060940.0,4189261.0,4208387.0,4175221.0,4164459.0,4175459.0,4150420.0,4141202.0,4222496.0,4272909.0,4203852.0,4207331.0,4225571.0,4271168.0,4379797.0,4478029.0,4555029.0,4680384.0,4770188.0,4761228.0,4572098.0,4461418.0,4382364.0,4396137.0,4425400.0,4282329.0,4340779.0,4330751.0,4278041.0,4358571.0,4084430.0,4002096.0,3958280.0,3840006.0,3954899.0,3827009.0,3879028.0,4049147.0,4275506.0,4324119.0,4091962.0,4000602.0,3998899.0,4066500.0,4303847.0,4373250.0,4360049.0,43

In [8]:
all_df = input_df.merge(pgmm,how="left",on=['Region', 'Country', 'State/Province', 'Date', 'Week'])
all_df["Output"] = all_df["Output"].apply(lambda x: x if x==True else False)
all_df["Level"] = all_df.apply(
    lambda x: "Country" if ((x["Country"]=="United States") & (x["State/Province"]=="")) else x["Level"],
    axis=1
)
all_order = [
    'Level', 'Region', 'Country', 'Census Region', 'State/Province', 'Abbreviation', 'FIPS',
    'Time','Date','Week','Last Day of Week Excel Date', 'Accessed',
    'Output', 'Status', 'Data Quality',
    'Speed', 'Speed Daily', 'Acceleration', 'Acceleration Daily', 'Jerk', 'Jerk Daily', '7-Day Persistence', '1-Day Persistence',
    'Cases Daily', 'Cases Daily 7D Rolling', 'Cases 7D Moving Average', 'Cases Daily Last Day of Week', 
    'Cases Daily Rate', 'Cases Daily Rate 7D Rolling', 'Cases Last Day of Week Rate 100K',
    'Total Cases', 'Total Cases Rate', 'Total Cases Last Day of Week',
    'Deaths Daily', 'Deaths Daily 7D Rolling', 'Deaths 7D Moving Average', 'Deaths Daily Last Day of Week', 
    'Deaths Daily Rate', 'Deaths Daily Rate 7D Rolling', 'Deaths Last Day of Week Rate 100K', 
    'Total Deaths', 'Total Deaths Rate', 'Total Deaths Last Day of Week',
    'Tests Daily', 'Tests Daily 7D Rolling',
    'Tests Daily Rate', 'Tests Daily Rate 7D Rolling',
    'Total Tests', 'Total Tests Rate',
    'Positivity 7D Rolling', 
    'Active Daily', 'Total Active', 
    'Negative Daily', 'Total Negative', 
    'Recovered Daily', 'Total Recovered', 
    'Hospitalized Daily', 'Currently Hospitalized', 'Total Hospitalized', 
    'ICU Daily', 'Total ICU', 
    'Currently In ICU', 'Total In ICU', 
    'Currently On Ventilator', 'Total On Ventilator', 
    "Population","Population 100K",
    "Country Population","Country Population 100K", "Country Share",
    "Region Population","Region Population 100K", "Region Share",
    "World Population", "World Population 100K", "World Share",
    'World Share (%)', 'Urban Population (%)', 'Annual Change (%)', 'Net Change', 'Migrants (net)', 'Density (P/Km²)',
    'Land Area (Km²)', 'Fertility Rate', 'Median Age',
    '< 1', 1, 2, 3, 4, 5, 6, 7, 8, 9,
    10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
    20, 21, 22, 23, 24, 25, 26, 27, 28, 29,
    30, 31, 32, 33, 34, 35, 36, 37, 38, 39,
    40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
    50, 51, 52, 53, 54, 55, 56, 57, 58, 59,
    60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
    70, 71, 72, 73, 74, 75, 76, 77, 78, 79,
    80, 81, 82, 83, 84, '85+',
    '1-4','5-14','15-24','25-34','35-44','45-54','55-64','65-74','75-84',
    'Pct < 1','Pct 1-4','Pct 5-14','Pct 15-24','Pct 25-34','Pct 35-44','Pct 45-54','Pct 55-64','Pct 65-74','Pct 75-84','Pct 85+'
]
print_column_missing(all_df.columns,all_order)
all_df = all_df[all_order]

all_out_file = cleanedFolder + "all_combined.xlsx"
print(all_out_file)
all_df.to_excel(all_out_file, index=False)
all_df.head(14)

Column Values:
['Population', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 'Active Daily', 'Total Deaths Rate', 'Last Day of Week Excel Date', 'Week', 'Time', 'Total Deaths Last Day of Week', 'Tests Daily Rate 7D Rolling', 'Pct 25-34', 'Deaths Daily Rate 7D Rolling', 'Positivity 7D Rolling', '25-34', 'Acceleration', 'Speed Daily', 'Pct 5-14', 'FIPS', 'Census Region', 'Density (P/Km²)', 'Urban Population (%)', 'Deaths Daily 7D Rolling', 'Cases Daily', 'Data Quality', 'Total Hospitalized', 'Cases Daily Rate 7D Rolling', 'Total Active', 'Total Recovered', 'Cases 7D Moving Average', 'Total Tests Rate', 'Negative Daily', 'Tests Daily', '35-44', '85+', 'Pct 35-44', 'Hospitalized Daily', 'Cases Daily 7D Rolling', 'P

,Level,Region,Country,Census Region,State/Province,Abbreviation,FIPS,Time,Date,Week,Last Day of Week Excel Date,Accessed,Output,Status,Data Quality,Speed,Speed Daily,Acceleration,Acceleration Daily,Jerk,Jerk Daily,7-Day Persistence,1-Day Persistence,Cases Daily,Cases Daily 7D Rolling,Cases 7D Moving Average,Cases Daily Last Day of Week,Cases Daily Rate,Cases Daily Rate 7D Rolling,Cases Last Day of Week Rate 100K,Total Cases,Total Cases Rate,Total Cases Last Day of Week,Deaths Daily,Deaths Daily 7D Rolling,Deaths 7D Moving Average,Deaths Daily Last Day of Week,Deaths Daily Rate,Deaths Daily Rate 7D Rolling,Deaths Last Day of Week Rate 100K,Total Deaths,Total Deaths Rate,Total Deaths Last Day of Week,Tests Daily,Tests Daily 7D Rolling,Tests Daily Rate,Tests Daily Rate 7D Rolling,Total Tests,Total Tests Rate,Positivity 7D Rolling,Active Daily,Total Active,Negative Daily,Total Negative,Recovered Daily,Total Recovered,Hospitalized Daily,Currently Hospitalized,Total Hospitalized,ICU Daily,Total ICU,Currently In ICU,Total In ICU,Currently On Ventilator,Total On Ventilator,Population,Population 100K,Country Population,Country Population 100K,Country Share,Region Population,Region Population 100K,Region Share,World Population,World Population 100K,World Share,World Share (%),Urban Population (%),Annual Change (%),Net Change,Migrants (net),Density (P/Km²),Land Area (Km²),Fertility Rate,Median Age,< 1,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85+,1-4,5-14,15-24,25-34,35-44,45-54,55-64,65-74,75-84,Pct < 1,Pct 1-4,Pct 5-14,Pct 15-24,Pct 25-34,Pct 35-44,Pct 45-54,Pct 55-64,Pct 65-74,Pct 75-84,Pct 85+
0,Country,Central Asia,Armenia,,,AM,NaN,2020-03-01,03/01/2020,2020 W09,NaN,12/29/2020,False,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,0.033747,NaN,NaN,1.0,0.033747,NaN,0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,0,NaN,0.000000,NaN,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2963243.0,29.63243,NaN,NaN,NaN,326887719.0,3268.87719,0.009065,7.796609e+09,77966.09105,0.00038,0.04,63.0,0.19,5512.0,-4998.0,104.0,28470.0,1.8,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Country,Central Asia,Armenia,,,AM,NaN,2020-03-02,03/02/2020,2020 W10,NaN,12/29/2020,False,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0.000000,NaN,NaN,1.0,0.033747,NaN,0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,0,NaN,0.000000,NaN,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2963243.0,29.63243,NaN,NaN,NaN,326887719.0,3268.87719,0.009065,7.796609e+09,77966.09105,0.00038,0.04,63.0,0.19,5512.0,-4998.0,104.0,28470.0,1.8,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Country,Central Asia,Armenia,,,AM,NaN,2020-03-03,03/03/2020,2020 W10,NaN,12/29/2020,False,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0.000000,NaN,NaN,1.0,0.033747,NaN,0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,0,NaN,0.000000,NaN,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2963243.0,29.63243,NaN,NaN,NaN,326887719.0,3268.87719,0.009065,7.796609e+09,77966.09105,0.00038,0.04,63.0,0.19,5512

In [ ]:
us_check = all_df.loc[(all_df["Country"]=="United States") & (all_df["State/Province"]=="") & (all_df["Output"]==True)]
us_check.head()